In [ ]:
from IPython.display import display, HTML

REQUIRED_VERSIONS = {
    "hbp_service_client": "1.1.0",
    'ipywidgets':'7.0.5'
}

def get_version(package_name):
    """Returns version for the given package"""
    if package_name=='hbp_service_client':
        import hbp_service_client
        return hbp_service_client.__version__
    if package_name=='ipywidgets':
        import ipywidgets
        return '.'.join(map(str, ipywidgets.version_info[0:3]))
    
def install_package(package_name, versions):
    """Updates the specified package."""
    print("Changing package {}".format(package_name))
    if package_name=='hbp_service_client':
        lib = "{}=={}".format(package_name, versions[package_name])
        !pip install -I --quiet $lib     
    if package_name=='ipywidgets':
        lib1 = 'ipywidgets=={}'.format(versions["ipywidgets"])
        versions["widgetsnbextension"]='3.0.8'
        lib2 = 'widgetsnbextension=={}'.format(versions["widgetsnbextension"])
        !pip2 install $lib1 $lib2
        !pip3 install $lib1 $lib2
        !jupyter nbextension uninstall --py widgetsnbextension
        !jupyter nbextension install --py --user widgetsnbextension
        !jupyter nbextension enable widgetsnbextension --user --py
    
# Check if a package needs to be changed
reload_required = []
for package_name, required_version in REQUIRED_VERSIONS.items():
    if get_version(package_name) != required_version:
        print("{} {}".format(get_version(package_name), required_version))
        reload_required.append(package_name)
        
if reload_required:
    display(HTML("""
    <div class="alert alert-block alert-danger">Some internal libraries had to be changed...<br>
    Please follow the popup to reload this page.</div>"""))
    
    # Change all package that requires change
    for package in reload_required:
        install_package(package, REQUIRED_VERSIONS)
    
    display(HTML("""<script>window.requestAnimationFrame(() => { Jupyter.notebook.kernel.restart(); \
    Jupyter.notebook.dirty = false; window.location.reload(); })</script>"""))
else:
    display(HTML("""
    <div class="alert alert-block alert-success">Setup for this notebook is correct.</div>"""))

In [ ]:
clients = get_hbp_service_client()
import os
if os.path.exists('34-200.png')==True:
    os.remove('34-200.png')
if os.path.exists('Manager.ipynb'):
    os.remove('Manager.ipynb')
clients.storage.download_file(os.path.join(os.path.sep,'9132',\
                                           '34-200.png'), '34-200.png');
clients.storage.download_file(os.path.join(os.path.sep,'9132',\
                                           'Manager_Python3.ipynb'), 'Manager_Python3.ipynb');
from IPython.display import HTML,Javascript, display
from ipywidgets import widgets
import requests
import xml.etree.ElementTree
import time
import sys
import unicodedata
display(HTML('''<style>
    .widget-label { min-width: 17ex !important;}
</style>'''))

javascript2 = """
<script type="text/Javascript">
document.styleSheets[0].insertRule(".widget-listbox.form-control { width: auto }", 1);
</script>
"""
h1=HTML(javascript2)
h1.display='none'
display(h1)
import warnings
warnings.filterwarnings("ignore")

username=''

def checkloginNSG(_):
    username=username_widget.value
    password=password_widget.value
    KEY = 'Application_Fitting-DA5A3D2F8B9B4A5D964D4D2285A49C57'
    headers = {'cipres-appkey' : KEY}
    URL = 'https://nsgr.sdsc.edu:8443/cipresrest/v1/job/'+username
    r = requests.get(URL, auth=(username, password),headers=headers)
    root = xml.etree.ElementTree.fromstring(r.text)
    a=0
    for child in root:
        if child.tag == 'displayMessage':
            a=1
            msg=child.text
    if a==0:
        msg='Authenticated successfully'
        msg_widget.disabled=True
        buttonlogin.disabled=True
        username_widget.disabled=True
        password_widget.disabled=True
    msg_widget.value=msg
    msg_widget.layout.display=''

username_widget = widgets.Text(description='Username:')
username_widget.layout.width='36%'
password_widget = widgets.Password(description='Password:')
buttonlogin = widgets.Button()
buttonlogin.description = 'Login NSG'
buttonlogin.background_color='gainsboro'
buttonlogin.border_color='black'
buttonlogin.width='20%'
buttonlogin.on_click(checkloginNSG)
msg_widget = widgets.Text()
msg_widget.layout.width='40%'
msg_widget.layout.display='none'
formMNSG=widgets.VBox(children=[username_widget,password_widget,buttonlogin,msg_widget])
formMNSG.layout.display=''
display(formMNSG)

def run_all(ev):
    global username, password
    username=username_widget.value
    password=password_widget.value
    button.layout.display='none'
    %run "Manager_Python3.ipynb"

button = widgets.Button(description="Job manager")
button.on_click(run_all)
display(button)

In [ ]:
from IPython.display import HTML
HTML('''
<form action="javascript:code_toggle()">
<input id="toggle-code-button" type="submit" value="Click here to toggle on/off the raw code."></form>
<script>
code_show=true;
console.log(code_show)
$('.widget-area .prompt .close').hide()
function code_toggle() {
 console.log(code_show)
 var list = document.getElementsByClassName('cell code_cell rendered unselected');
 var list_index = []
 for (var i = 0; i < list.length; i++) {
    var curr_des = [];
    curr_wrapper = $(list[i]).find(".output_wrapper");
    curr_area = $(list[i]).find(".output_area");
    if (curr_wrapper.length > 0 && curr_area.length == 0){
    list_index.push(i)
    } else if (curr_wrapper.length == 0 && curr_area.length == 0){
    list_index.push(i)
    };
 }
 console.log(list.length)
 console.log(list_index)
 for (var j = 0; j< list_index.length; j++){
 console.log(list[list_index[j]].style.display)
 }
 if (code_show){
 $('div.input').hide();
 for (var j = 0; j< list_index.length; j++){
    list[list_index[j]].style.display = "none"
    }
 } else {
 $('div.input').show();
 for (var j = 0; j< list_index.length; j++){
    list[list_index[j]].style.display = "block"
    }
 }
 code_show = !code_show
 console.log(code_show)

 }
document.getElementById("toggle-code-button").click(); 
</script>
''')